In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.pagina12.com.ar/"

In [3]:
p12 = requests.get(url)

In [4]:
s = BeautifulSoup(p12.text, 'lxml')

In [5]:
secciones = s.find('ul',attrs={'class':'horizontal-list main-sections hide-on-dropdown'}).find_all('li')

In [6]:
seccion = secciones[0]

In [7]:
seccion.a

<a href="https://www.pagina12.com.ar/secciones/el-pais">El país</a>

In [8]:
seccion.a.get('href')

'https://www.pagina12.com.ar/secciones/el-pais'

In [9]:
links_secciones = [seccion.a.get('href') for seccion in secciones]
links_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/secciones/dialogos']

print(s.prettify())

In [10]:
sec = requests.get(links_secciones[0])
sec.status_code

200

In [11]:
sec

<Response [200]>

In [12]:
s_seccion = BeautifulSoup(sec.text, 'lxml')
#print(s_seccion.prettify())

In [13]:
s_seccion.find('div',attrs={'class': 'article-item__content'})

<div class="article-item__content"><!-- Title --><h2 class="title-list"><a href="https://www.pagina12.com.ar/310568-oscar-parrilli-el-lawfare-sigue-vivito-y-coleando">Oscar Parrilli: “El Lawfare sigue vivito y coleando”</a></h2><!-- Kicker --><p><a href="https://www.pagina12.com.ar/310568-oscar-parrilli-el-lawfare-sigue-vivito-y-coleando">Definiciones del senador del Frente de Todos</a></p></div>

In [14]:
s_seccion.find('div',attrs={'class': 'article-item__content'}).a.get('href')

'https://www.pagina12.com.ar/310568-oscar-parrilli-el-lawfare-sigue-vivito-y-coleando'

In [15]:
article_list = s_seccion.find('div', attrs={'class': 'articles-list'})

In [16]:
def notices(url):
    articles_list = []
    
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')

    secciones = s.find('ul',attrs={'class':'horizontal-list main-sections hide-on-dropdown'}).find_all('li')
    links_secciones = [seccion.a.get('href') for seccion in secciones]
    
    for article in links_secciones:
        sec = requests.get(article)
        soup_n = BeautifulSoup(sec.text, 'lxml')
        
        arti_link = soup_n.find('div',attrs={'class': 'article-item__content'})
        if arti_link:
            arti_link = arti_link.find('h2', attrs={'class': 'title-list'})
            articles_list.append(arti_link.a.get('href'))
    
        
        arti_link = soup_n.find('div', attrs={'articles-list is-grid-col2 grid-mobile-row'})
        if arti_link:
            articles_list.append(arti_link.find('div',attrs={'class': 'article-item__header'}).a.get('href'))
        
        
        arti_link = soup_n.find_all('div', attrs={'class':'articles-list'})
        if arti_link:
            arti_link = arti_link[1].find_all('div', attrs={'class':'article-item__header'})
            for arti in arti_link:
                articulito = arti.a.get('href')
                articles_list.append(articulito)
    
    
    return articles_list
    
url = "https://www.pagina12.com.ar/"
urls= notices(url)

In [17]:
len(urls)

60

In [18]:
url_mala = url.replace('2','3')

In [19]:
try:
    r = requests.get(url_mala)
except Exception as e:
    print("error: {}\n".format(e))
    

error: HTTPSConnectionPool(host='www.pagina13.com.ar', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000253CE5D1708>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))



In [20]:
url_nota = urls[0]
url_nota

'https://www.pagina12.com.ar/309681-los-referentes-del-frente-de-todos-en-la-ciudad-defendieron-'

In [54]:
def extract_info(urls):
    
    titles =[]
    dates  =[]
    intros =[]
    bodies =[]
    authors=[]
    
    for url_single in urls:
        nota = requests.get(url_single)
        if str(nota.status_code)=="200":
            s_nota = BeautifulSoup(nota.text, 'lxml')
            #print("Añadiendo noticia de: {}".format(url_single))
        #Extract title
            title = s_nota.find('div', attrs={'class':'article-titles'}).h1
            if title:
                titles.append(title.text.replace('\xa0',' '))
            else: titles.append([])
        #Exract date
            date = s_nota.find('div', attrs={'class':'article-info'})
            dates.append(date.find('div', attrs={'time'}).span.get('datetime'))
         #Extract intro
            intro = s_nota.find('div', attrs={'class':'article-titles'}).div
            if intro:
                intros.append(intro.text)
            else:
                intros.append([])

            ##Extract body
                #This one is more difficult due to the html structure
            body = s_nota.find('div', attrs={'class':'article-text'}).find_all('p')
            body_total = []

            for element in body:
                body_total.append(str(element).replace('<p>', '').replace('</p>','').replace('<b>', '').replace('</b>','').replace('<br/>','').replace('&amp;','&'))
                body_total.append('\n\n')
                #Finally, we join all the elements and add intro spaces
            bodies.append(''.join(body_total) )

            # Extract Author
            author = s_nota.find('div', attrs={'class':'article-main-media'}).find('div', attrs={'class':'article-author'})
            if author:
                authors.append(author.a.text)
            else: authors.append([])
        
            
    return titles, dates, intros, bodies, authors

In [55]:
titles, dates, intros, bodies, authors = extract_info(urls)

In [56]:
print(urls[1])
print(titles[1])
#print(intros[1])
#print(bodies[1])
#print(dates[1])

https://www.pagina12.com.ar/309679-alberto-fernandez-recibio-a-llamosas-fue-el-resultado-de-la-
Alberto Fernández recibió a Llamosas: "Fue el resultado de la unidad"


In [ ]:
url = 'https://www.pagina12.com.ar/309358-rodriguez-larreta-hace-campana-y-apuesta-todo-a-la-corte-sup'
p12 = requests.get(url)
s = BeautifulSoup(p12.text, 'lxml')

In [ ]:
media = s.find('div', attrs={'class':'article-main-media-image'})
imagenes = media.find_all('img')

if len(imagenes)==0:
    print("NO se encontraron imágenes")
else:
    image = imagenes[-1]
    img_src = image.get('data-src')
    print(img_src)

In [ ]:
img_req = requests.get(img_src)
img_req.status_code

In [26]:
from IPython.display import Image

In [ ]:
Image(img_req.content)

In [27]:
import pandas as pd

In [57]:
data = {'title': titles,
        'author': authors,
        'date': dates,
        'intro': intros,
        'body': bodies,
        'url': urls
       }

In [60]:
df = pd.DataFrame(data)

In [66]:
df.loc(1)